In [90]:
import tkinter as tk
from tkinter import ttk
import random

people =  0
size =0
tables = [None] * 12
num_initial_guests = 12 * 6 #模擬已經客滿，6人桌*12
initial_groups = []
group_id = 1
for i in range(len(tables)):
    table = []
    remaining_seats = 6
    while remaining_seats > 0:
        group_size = random.randint(1, min(4, remaining_seats))  # 每組客人1到4人
        leave_time = random.randint(2, 7)  #初始客人隨機離開
        table.append((group_id, group_size, 0, leave_time))
        initial_groups.append((group_id, group_size, 0, leave_time))
        group_id += 1
        remaining_seats -= group_size
    tables[i] = table

arrival_times = []#新客人
group_sizes = []
current_time = 0
while people < 300:  # 新來的人數共300人
    current_time += 1 
    for _ in range(random.randint(4,6)):#每個時間點來4到6組客人
        arrival_times.append(current_time)
        size = random.randint(1, 4)
        group_sizes.append(size)
        people += size
arrival_times = [0] * len(initial_groups) + arrival_times #合併初始&後到客人
group_sizes = [group[1] for group in initial_groups] + group_sizes

waiting_queue = []
seated_groups = initial_groups[:]


root = tk.Tk()#建立視窗
root.title('餐廳座位模擬')

canvas = tk.Canvas(root, width=600, height=800)#餐廳顯示範圍
canvas.pack(side=tk.LEFT)
# 文字部分顯示設定
waiting_label = tk.Label(root, text="等待中：", justify=tk.LEFT)
waiting_label.pack(side=tk.RIGHT, padx=10)
change_log_label = tk.Label(root, text="入座安排：\n", justify=tk.LEFT)
change_log_label.pack(side=tk.RIGHT, padx=10, pady=10)
#expected_wait_time_label = tk.Label(root, text="預計等待時間：\n", justify=tk.LEFT) #預計等待時間
#expected_wait_time_label.pack(side=tk.RIGHT, padx=10, pady=10)
total_wait_time_label = tk.Label(root, text="等待時間總和：0\n已用餐人數：0\n入座平均等待時間：{total_wait_time/seated_people}", justify=tk.LEFT)
total_wait_time_label.pack(side=tk.BOTTOM, pady=10)
time_label = tk.Label(root, text=f"時間：0", justify=tk.LEFT)#顯示時間
time_label.pack(side=tk.TOP, pady=10)
button_frame = tk.Frame(root)#新增按鈕控制時間
button_frame.pack(side=tk.BOTTOM, pady=10)
next_button = ttk.Button(button_frame, text="下一個時間點", command=lambda: change_time(1))
next_button.pack(side=tk.RIGHT, padx=10, pady=6)
current_time = 0

def simulate():#圖形界面
    global current_time, waiting_queue, tables, seated_groups, changed_tables, changed_seats
    canvas.delete("all")
    #設定桌子
    y_offset = 50
    for i, table in enumerate(tables):
        x_offset = 50 + (i % 3) * 200  #調整x間距
        if i % 3 == 0 and i != 0:
            y_offset += 180  # 調整y間距
        table_text = f"第{i+1:3}桌：\n"
        seat_count = 0
        if table is None or len(table) == 0:#空位
            table_text += "空"
            fill_color = "light yellow"
            for j in range(6):
                seat_x = x_offset + (j % 3) * 50  
                seat_y = y_offset + (j // 3) * 40  
                canvas.create_rectangle(seat_x, seat_y, seat_x + 50, seat_y + 40, fill=fill_color)
                canvas.create_text(seat_x + 25, seat_y + 20, text="空")
        else:
            for group in table:#顯示入座情況
                table_text += f"第{group[0]:4}組 {group[1]:3}人\n"
                for _ in range(group[1]):
                    seat_x = x_offset + (seat_count % 3) * 50  
                    seat_y = y_offset + (seat_count // 3) * 40  
                    if (i, seat_count) in changed_seats:
                        fill_color = "light green"
                    else:
                        fill_color = "white"
                    canvas.create_rectangle(seat_x, seat_y, seat_x + 50, seat_y + 40, fill=fill_color)
                    canvas.create_text(seat_x + 25, seat_y + 20, text=f"{group[0]}")
                    seat_count += 1
            for j in range(seat_count, 6):#離座後空桌
                seat_x = x_offset + (j % 3) * 50  
                seat_y = y_offset + (j // 3) * 40  
                canvas.create_rectangle(seat_x, seat_y, seat_x + 50, seat_y + 40, fill="light yellow")
                canvas.create_text(seat_x + 25, seat_y + 20, text="空")
        canvas.create_text(x_offset + 75, y_offset + 140, text=table_text, font=('Arial', 10))  #顯示桌號

    waiting_text = "等待中：\n"#文字部分
    total_wait_time = 0
    seated_people = 0
    for group in seated_groups:#計算已入座組別的等待時間
        seated_people += group[1]
        if group[2] > 0:
            total_wait_time += (group[2] - group[4]) * group[1]
    for group in waiting_queue:# 計算未入座組別的等待時間
        if group[2] <= current_time:
            waiting_text += f"第{group[0]:4}組 {group[1]:3}人 - 等待時間 {current_time - group[2]}\n"#將還沒入座的組別顯示在等待列
            total_wait_time += (current_time - group[2]) * group[1]
    waiting_label.config(text=waiting_text)
    if seated_people > 72:
        total_wait_time_label.config(text=f"等待時間總和：{total_wait_time}\n已用餐人數：{seated_people-72}\n入座平均等待時間：{total_wait_time/(seated_people-72):.4f}", font=16)
    else:
        total_wait_time_label.config(text=f"等待時間總和：{total_wait_time}\n已用餐人數：{seated_people-72}\n入座平均等待時間：0", font=16)
    time_label.config(text=f"現在時間：{current_time}")
    # 顯示預計等待時間
    #expected_wait_times = calculate_expected_wait_times()
    #expected_wait_text = "預計等待時間：\n"
    #for size, wait_time in expected_wait_times.items():
    #    expected_wait_text += f"{size} 人：{wait_time} 分鐘\n"
    #expected_wait_time_label.config(text=expected_wait_text)
    update_change_log()#更新變化
    
def change_time(delta):#時間
    global current_time, changed_tables, changed_seats
    current_time += delta
    current_time = max(0, current_time)
    changed_tables, changed_seats = update_seating()
    simulate()
def update_seating():
    global current_time, waiting_queue, tables, seated_groups
    changed_tables = set()
    changed_seats = set()
    for i in range(len(arrival_times)):#新客人加入排隊
        if arrival_times[i] == current_time:
            group_id = i + 1
            group_size = group_sizes[i]
            arrival_time = arrival_times[i]
            waiting_queue.append((group_id, group_size, arrival_time))
    for i in range(len(tables)):#處理離座的客人
        if tables[i]:
            new_table = []
            seat_count = 0
            for group in tables[i]:
                if group[3] == current_time:
                    for _ in range(group[1]):
                        changed_seats.add((i, seat_count))
                        seat_count += 1
                    changed_tables.add(i)
                else:
                    new_table.append(group)
                    seat_count += group[1]
            tables[i] = new_table if new_table else None    
    new_waiting_queue = []   #嘗試安排座位
    for group in waiting_queue:
        group_id, group_size, arrival_time = group
        seated = False
        min_wait_time = float('inf')#尋找等待時間最小的安排
        best_table_index = None
        best_position = None

        for i in range(len(tables)):
            if tables[i] is None:
                min_wait_time = 0
                best_table_index = i
                best_position = 0
                break
            else:
                total_seats = sum(g[1] for g in tables[i])
                if total_seats + group_size <= 6:
                    potential_wait_time = current_time - arrival_time
                    if potential_wait_time < min_wait_time:
                        min_wait_time = potential_wait_time
                        best_table_index = i
                        best_position = total_seats
        if best_table_index is not None:
            dining_time = random.randint(5, 8)
            if tables[best_table_index] is None:
                tables[best_table_index] = [(group_id, group_size, current_time, current_time + dining_time, arrival_time)]
            else:
                tables[best_table_index].append((group_id, group_size, current_time, current_time + dining_time, arrival_time))
            seated_groups.append(tables[best_table_index][-1])
            for j in range(best_position, best_position + group_size):
                changed_seats.add((best_table_index, j))
            changed_tables.add(best_table_index)
            seated = True
        if not seated:
            new_waiting_queue.append(group)
    waiting_queue[:] = new_waiting_queue
    return changed_tables, changed_seats #回傳更新

#def calculate_expected_wait_times():#計算預計等待時間
#    wait_times = {1:0,2:0,3:0,4:0}
#    counts = {1:0,2:0,3:0,4:0}
#    for table in tables:
#        if table:
#            for group in table:
#                size = group[1]
#                wait_times[size] += group[3] - current_time
#                counts[size] += 1
#    expected_wait_times = {}
#    for size in wait_times:
#        if counts[size] > 0:
#            expected_wait_times[size] = wait_times[size] // counts[size]
#        else:
#            expected_wait_times[size] = 0
#    return expected_wait_times

def update_change_log():#顯示時間點入座安排
    global change_log_label, changed_tables, changed_seats
    change_log_text = "入座安排：\n"
    for table_index in changed_tables:
        groups = tables[table_index]
        if groups:
            for group in groups:
                if group[2] == current_time:  
                    change_log_text += f"第{group[0]:4}組 {group[1]:3}人入座第{table_index+1}桌\n" 
    change_log_label.config(text=change_log_text)

    
    
    
#初始化
changed_tables = set()
changed_seats = set()
simulate()
root.mainloop()
